In [106]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [107]:
bn = gum.BayesNet('FraudDetection')

In [108]:
# Adding variables 

transaction_info = bn.add(gum.LabelizedVariable('TransactionInfo', 'transactioninfo', ['suspicious', 'normal']))

transaction_amount = bn.add(gum.LabelizedVariable('TransactionAmount', 'transactionamount', ['low', 'medium', 'high']))
transaction_frequency = bn.add(gum.LabelizedVariable('TransactionFrequency', 'transactionfrequency', ['low', 'medium', 'high']))

geographic_location = bn.add(gum.LabelizedVariable('GeographicLocation', 'geographiclocation', ['local', 'domestic', 'international']))

account_age = bn.add(gum.LabelizedVariable('AccountAge', 'accountage', ['new', 'established']))
user_behavior = bn.add(gum.LabelizedVariable('UserBehavior', 'userbehavior', ['normal', 'abnormal']))

fraud_detected = bn.add(gum.LabelizedVariable('FraudDetected','frauddetected', ['yes','no']))
print ("BN With vars : ",bn)

BN With vars :  BN{nodes: 7, arcs: 0, domainSize: 432, dim: 10, mem: 136o}


In [109]:
# Define the arcs between variables
arcs = [
    (transaction_amount, transaction_info),
    (transaction_frequency, transaction_info),
    
    (transaction_info,fraud_detected),
    
    (user_behavior, fraud_detected),
    (account_age,user_behavior),
    
    (geographic_location, fraud_detected),
]
# Add the arcs to the Bayesian network
for arc in arcs:
    bn.addArc(*arc)
bn

(pyAgrum.BayesNet<double>@0x7f8d3d81d000) BN{nodes: 7, arcs: 6, domainSize: 432, dim: 30, mem: 456o}

In [110]:
# Define the conditional probability tables 
#bn.cpt('ClinicalSymptoms').fillWith([0.25,0.35,0.4])
# bn.cpt('Gender').fillWith([0.55,0.45])
# #bn.cpt('MedicalHistory').fillWith([0.55,0.35,0.1])
# bn.cpt('TreatmentCompliance').fillWith([0.3,0.7])
#bn.cpt('UrbanLocation').fillWith([0.2,0.8])

bn.cpt('AccountAge').fillWith([0.5,0.5])
bn.cpt('GeographicLocation').fillWith([0.45,0.35,0.2])

bn.cpt('TransactionFrequency').fillWith([0.5,0.35,0.15])
bn.cpt('TransactionAmount').fillWith([0.45,0.35,0.2])

bn.cpt('TransactionInfo')[{'TransactionFrequency': 0, 'TransactionAmount': 0}] = [0.85, 0.15]
bn.cpt('TransactionInfo')[{'TransactionFrequency': 0, 'TransactionAmount': 1}] = [0.5, 0.35]
bn.cpt('TransactionInfo')[{'TransactionFrequency': 1, 'TransactionAmount': 0}] = [0.1, 0.9]
bn.cpt('TransactionInfo')[{'TransactionFrequency': 1, 'TransactionAmount': 1}] = [0.05, 0.95]

bn.cpt('UserBehavior')[{'AccountAge': 1}] = [0.6, 0.4]
bn.cpt('UserBehavior')[{'AccountAge': 0}] = [0.45, 0.55]


In [111]:
# Showing the BN
bn
#bn.saveBIFXML("BN_model.bif")

(pyAgrum.BayesNet<double>@0x7f8d3d81d000) BN{nodes: 7, arcs: 6, domainSize: 432, dim: 30, mem: 456o}

In [112]:
#Compute the posterior probability of TB Infection - inference
#gum.makeInference()
ie = gum.VariableElimination(bn)
gnb.showInference(bn, evs={}, engine=ie)


#Compile the Bayesian network 
ie = gum.LazyPropagation(bn)


#Set the evidence (observed nodes)
#ie.setEvidence({'UrbanLocation': 'rural'})


Exception: [pyAgrum] Several evidence/CPT are incompatible together (proba=0): some evidence entered into the Bayes net are incompatible (their joint proba = 0)